# Model Context Protocol: Connecting AI Agents to the World
As our code review agent grows more sophisticated, we face a challenge; how do we connect it to the growing ecosystem of external tools and data sources.
So far, our agent's tools are hardcoded Python functions. Every time we want to add a new capability, like, access to a database, integrations with a project management system or connection to a version control system, we need to write custom code.

The **Model Context Protocol (MCP)** solves this problem by providin a standardized way for AI applications to connect to data sources and tools.  
Instead of building custom integrations for every external system, MCP provides an universal protocol.

Model Context Protocol is an open protocol developed by [Anthropic](https://www.anthropic.com/news/model-context-protocol) that standardizes how AI applications connect to external systems. It defines:
* **A client-server architecture:** where your agent (the client) connects to MCP servers that expose tools and data
* **A standard message format:** using [JSON-RPC 2.0](https://www.jsonrpc.org/specification) for communication
* **MCP architecture primitives:** Resources(data), Tools(actions), and Prompts(templates)

## MCP Architecture: Core Concepts

### The Three Primitives
MCP defines three types of capabilities that servers can expose:

#### 1. Resources
Resources are **data sources** that an agent can read. These could be files, database records or documents the agent can access

```
Example Resources:
- File contents: file:///home/user/project/
- Database records: db://localhost/project_db/users
- API responses: github://anthropics/repo/issues
```
Resources are identified by URIs and can be:
- **Listed:** "What resources are available"
- **Read:** "Give me the contents of this resource"
- **Subscribed to:** "Notify me when this resource changes"

#### 2. Tools
Tools are **actions** that an agent can execute. They are like the functions our agent already uses, but standardized.

```
Example Tools:
- create_github_issue(title, body)
- query_database(sql)
- send_email(to, subject, body)
```
Tools have
- **Names:** Unique identifiers
- **Descriptions:** What the tool does (This helps the LLM choose)
- **Input Schemas:** What parameters they accept - [JSON Schema](https://json-schema.org/docs)
- **Outputs:** Results of the action

#### 3. Prompts
Prompts are **templated interactions** that servers can provide. They are like reusable prompt snippets with variables.

```
Example Prompts:
- code_review: "Review this code: {{code}}"
- bug_report: "Report a bug in {{file}} at line {{line}}"
- summarize_pr: "Summarize PR #{{number}}"
```

### Client=Server Architecture
#### The Agent (Client)
- Discovers what servers are available
- Requests resources and tool definitions
- Calls tools through the protocol
- Processes results

#### The Server
- Exposes capabilities (resources, tools, prompts)
- Handles requests from clients
- Manages authentication and permissions
- Returns structured responses

### Communication Protocol
MCP uses JSON-RPC 2.0 over standard transport layer (stdio, HTTP, or WebSocket):

```json
// Client asks: "What tools do you have"
{
    "jsonrpc": "2.0",
    "method": "tools/list",
    "params": {},
    "id": 1
}

// Server responds: "Here are my tools"
{
    "jsonrpc":"2.0",
    "result": {
        "tools": [
            {
                "name":"read_file",
                "description": "Read file contents",
                "inputSchema": {
                    "type": "object",
                    "properties": {
                        "path": {"type":"string"}
                    }
                }
            }
        ]
    },
    "id": 1
}
```

## Building our own MCP Server
Let's convert our code review tools into an MCP Server. This will make them usable by any MCP compatible client.

We will be making changes to this code [CodeReviewAgentWithTools](https://github.com/asanyaga/ai-agents-tutorial/blob/main/code_review_agent_with_tools.ipynb)

### Setting up the MCP SDK
Install the MCP Python SDK

```bash
pip install mcp
```
## Creating a Basic MCP Server

Create a seperate file called code_review_mcp_server.py to run the server. Add the following code

```python
from mcp.server import Server
import mcp.server.stdio

# Create the server instance
server = Server("code-review-server")

# We'll add capabilities here

# Run the server over stdio
async def main():
    async with mcp.server.stdio.stdio_server() as (read_stream, write_stream):
        await server.run(
            read_stream,
            write_stream,
            server.create_initialization_options()
        )

if __name__ == "__main__":
    import asyncio
    asyncio.run(main())
```

### Converting read_file to an MCP Resource
Resources represent data that can be read. The ```read_file``` function is a perfect candidate
Update the server code with the following

```python
from mcp.server import Server
from mcp.types import Tool, TextContent, Resource
import mcp.server.stdio
import asyncio
import os
# Create the server instance
server = Server("code-review-server")

# Define a resource template for files

@server.list_resources()
async def list_resources() -> list[Resource]:
    """List available file resources in the current directory"""
    resources = []
    for root, dirs, files in os.walk("."):
        for file in files:
            if file.endswith(".py"):
                file_path = os.path.join(root, file)
                resources.append(
                    Resource(uri=f"file://{os.path.abspath(file_path)}",
                             name="file",
                             description=f"Python file: {file_path}",
                             mimeType="text/x-python")
                )
    return resources
@server.read_resource()
async def read_resource(uri: str) -> str:
    """Read the contents of a file resource"""
    # Extract the file path from the uri
    if uri.startswith("file://"):
        file_path = uri[:7] # Remove "file//:" prefix

        if not os.path.exists(file_path):
            raise ValueError(f"File not found: {file_path}")
        
        with open(file_path,"r") as f:
            content = f.read()
        
        return TextContent(
            type="text",
            text=content
        )

    raise ValueError(f"Usupported URI {uri}")
```

**What changed**
- ```read_file(path)``` becomes an MCP resource with a URI scheme
- Files are **discovered** through ```list_resources```
- Files are ***accessed** through ```read_resource(uri)```
- Resources are self describing with metadata (name, type, description)

### Converting Tools to MCP Tools
Now, let's convert our analyze_code tool

Update the MCP server code with the following
```python
from mcp.types import Tool, TextContent, Resource

@server.list_tools()
async def list_tools() -> list[Tool]:
    """Declare available tools"""
    return [
        Tool(
            name="analyze_code",
            description="Analyze Python code and provide imrovement suggestions",
            inputSchema={
                "type":"object",
                "properties": {
                    "code" : {
                        "type":"string",
                        "description": "The Python code to analyze"
                    }
                }
            }
        )
    ]

@server.call_tool()
async def call_tool(name: str, arguments: dict) -> list[TextContent]:
    if name == "analyze_code":
        code = arguments["code"]
        result = analyze_code_impl(code)

        return [TextContent(type="text",text=result)]
        
# Implementation functions (existing code)
def analyze_code_impl(code: str) -> str:
    """Implementation of code analysis"""
    import openai
    prompt = f"""
    You are a helpful code review assistant.
    Analyze the following Python code and suggest one improvement.

    Code:
    {code}
    """
    response = openai.responses.create(
        model="gpt-4.1-mini",
        input=[{"role": "user", "content": prompt}]
    )
    return response.output_text
```

#### what changed
- Tools are declared with JSON schema for their inputs
- Tools are invoked through a standard ```call_tool``` dispatcher
- Tool metadata (description, parameters) helps LLMs choose the right tool
- Results are wrapped in standard MCP response types

## Converting the agent to an MCP client
Now let's update our agent to MCP servers instead of using hardcoded tools.

### Create an MCP Client Manager
Let's create a manager that handles MCP server connections

Create a file ```run_mcp_agent.py`` with the following code

```run_mcp_agent.py```

```python
"""
MCP Code Review Agent - Standalone Script
Run: python run_mcp_agent.py
"""

import asyncio
from contextlib import AsyncExitStack
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from typing import Optional
import json
import openai
import os

class MCPClientManager:
    """Official MCP Client Manager using AsyncExitStack"""
    
    def __init__(self):
        self.session: Optional[ClientSession] = None
        self.exit_stack = AsyncExitStack()
        self.available_tools: dict[str, dict] = {}
    
    async def connect_to_server(self, server_script_path: str):
        """Connect to an MCP server"""
        is_python = server_script_path.endswith('.py')
        is_js = server_script_path.endswith('.js')
        if not (is_python or is_js):
            raise ValueError("Server script must be a .py or .js file")
        
        command = "python" if is_python else "node"

        server_env = {"OPENAI_API_KEY": os.getenv("OPENAI_API_KEY")}


        server_params = StdioServerParameters(
            command=command,
            args=[server_script_path],
            env=server_env
        )
        
        print(f"🔌 Connecting to MCP server: {server_script_path}")
        
        # Use AsyncExitStack (official pattern)
        stdio_transport = await self.exit_stack.enter_async_context(
            stdio_client(server_params)
        )
        stdio, write = stdio_transport
        
        self.session = await self.exit_stack.enter_async_context(
            ClientSession(stdio, write)
        )
        
        await self.session.initialize()
        
        # Discover tools
        response = await self.session.list_tools()
        for tool in response.tools:
            self.available_tools[tool.name] = {"tool": tool}
            print(f"Discovered tool: {tool.name}")
        
        print(f"Connected successfully!\n")
    
    async def call_tool(self, tool_name: str, arguments: dict) -> str:
        """Call a tool through MCP"""
        if not self.session:
            raise RuntimeError("Not connected to a server")
        
        if tool_name not in self.available_tools:
            raise ValueError(f"Unknown tool: {tool_name}")
        
        result = await self.session.call_tool(tool_name, arguments)
        
        if result.content and len(result.content) > 0:
            return result.content[0].text
        
        return "Tool executed successfully (no output)"
    
    def get_tool_descriptions(self) -> list[dict]:
        """Get tool descriptions for the LLM"""
        return [
            {
                "name": tool_name,
                "description": tool_info["tool"].description,
                "input_schema": tool_info["tool"].inputSchema
            }
            for tool_name, tool_info in self.available_tools.items()
        ]
    
    async def cleanup(self):
        """Clean up resources"""
        print("\nCleaning up...")
        await self.exit_stack.aclose()
        print("Cleanup complete!")

class CodeReviewAgentMCP:
    """Code review agent powered by MCP"""
    
    def __init__(self, mcp_manager: MCPClientManager, model="gpt-4o-mini"):
        self.mcp = mcp_manager
        self.model = model
    
    async def think(self, user_input: str) -> str:
        """LLM decides which tool to use"""
        tool_descriptions = self.mcp.get_tool_descriptions()
        tools_list = "\n".join([
            f"- {tool['name']}: {tool['description']}"
            for tool in tool_descriptions
        ])
        
        prompt = f"""
        You are a code assistant with access to these tools:
        
        {tools_list}
        
        Based on the user request, decide which tool to use.
        Reply ONLY with JSON: {{"tool": "tool_name", "args": {{"param": "value"}}}}
        
        Example: {{"tool": "analyze_code", "args": {{"code": "def foo(): pass"}}}}
        
        User request: {user_input}
        """
        
        response = openai.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}]
        )
        return response.choices[0].message.content
    
    async def act(self, decision: str) -> str:
        """Execute the chosen tool"""
        try:
            parsed = json.loads(decision)
            tool_name = parsed["tool"]
            args = parsed.get("args", {})
            
            result = await self.mcp.call_tool(tool_name, args)
            return result
        except Exception as e:
            return f"Error executing tool: {e}"
    
    async def run(self, user_input: str) -> str:
        """Complete think-act loop"""
        print(f"Thinking about: {user_input}")
        decision = await self.think(user_input)
        print(f"Decision: {decision}")
        
        print(f"\nExecuting...")
        result = await self.act(decision)
        print(f"\nResult:\n{result}")
        
        return result


async def main():
    """Main execution function"""
    print("=" * 60)
    print("MCP Code Review Agent")
    print("=" * 60)
    print()
    
    # Check for OpenAI API key
    if not os.getenv("OPENAI_API_KEY"):
        print("Warning: OPENAI_API_KEY environment variable not set")
        print("Please set it with: export OPENAI_API_KEY='your-key-here'")
        return
    
    mcp_manager = MCPClientManager()
    
    try:
        # Connect to MCP server
        await mcp_manager.connect_to_server("code_review_mcp_server.py")
        
        # Create agent
        agent = CodeReviewAgentMCP(mcp_manager)
        
        # Test with a code snippet
        code_snippet = """
def divide(a, b):
    return a / b
        """
        
        user_request = f"Please analyze this code: {code_snippet}"
        result = await agent.run(user_request)
        
        print("\n" + "=" * 60)
        print("Demo Complete!")
        print("=" * 60)
        
    except FileNotFoundError:
        print("Error: code_review_mcp_server.py not found")
        print("Make sure the MCP server file is in the same directory")
    except Exception as e:
        print(f"Error: {e}")
        import traceback
        traceback.print_exc()
    finally:
        await mcp_manager.cleanup()


if __name__ == "__main__":
    # Run the async main function
    asyncio.run(main())
```